In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=25)

PATH='data/favorita/'

In [3]:
!ls {PATH}

holidays_events.csv	       sample_submission.csv
items.csv		       stores.csv
joined			       test.csv
joined_test		       train.csv
joined_test_with_stores_items  train_one_year_full_data
joined_with_stores_items       transactions.csv
oil.csv


In [4]:
from IPython.display import HTML

In [5]:
dtypes = {'store_nbr': np.dtype('int64'),
          'item_nbr': np.dtype('int64'),
          'unit_sales': np.dtype('float64'),
          'onpromotion': np.dtype('bool')}

In [6]:
train = pd.read_csv(f'{PATH}train.csv', index_col='id', parse_dates=['date'], low_memory=False, dtype=dtypes)

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
# Replace onpromotion NaN values w/ False
# train.fillna(False, inplace=True)

In [7]:
display(train.head(), 'TRAIN')

,date,store_nbr,item_nbr,unit_sales,onpromotion
id,,,,,
0,2013-01-01,25,103665,7.0,NaN
1,2013-01-01,25,105574,1.0,NaN
2,2013-01-01,25,105575,2.0,NaN
3,2013-01-01,25,108079,1.0,NaN
4,2013-01-01,25,108701,1.0,NaN


'TRAIN'

In [8]:
date_mask = (train['date'] >= '2017-02-15') & (train['date'] <= '2017-08-15')
# results in a list of indexes and boolean value => w/in date range specified

In [9]:
len(train[date_mask]), len(train)

(19193423, 125497040)

In [10]:
# full_train = train
train = train[date_mask]

In [11]:
table_names = ['stores', 'transactions', 'items', 'oil', 'holidays_events', 'test']

In [12]:
tables = [pd.read_csv(f'{PATH}{fname}.csv', low_memory=False) for fname in table_names]

In [13]:
for i in range(0, len(table_names)): display(tables[i].head(), table_names[i], len(tables[i]))
# for t in tables: display(t.head()), display(table_names[t.index])

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


'stores'

54

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


'transactions'

...

'holidays_events'

350

,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,False
1,125497041,2017-08-16,1,99197,False
2,125497042,2017-08-16,1,103501,False
3,125497043,2017-08-16,1,103520,False
4,125497044,2017-08-16,1,103665,False


'test'

3370464

In [14]:
stores, transactions, items, oil, holidays_events, test = tables

### Convert holidays_events to local / regional / national

In [19]:
stores.city.unique()

array(['Quito', 'Santo Domingo', 'Cayambe', 'Latacunga', 'Riobamba', 'Ibarra', 'Guaranda', 'Puyo', 'Ambato',
       'Guayaquil', 'Salinas', 'Daule', 'Babahoyo', 'Quevedo', 'Playas', 'Libertad', 'Cuenca', 'Loja',
       'Machala', 'Esmeraldas', 'Manta', 'El Carmen'], dtype=object)

In [34]:
holidays_events.head(15)

,date,type,locale,locale_name,description,transferred,holiday
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,True
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,True
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,True
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,True
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,True
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False,True
6,2012-06-23,Holiday,Local,Guaranda,Cantonizacion de Guaranda,False,True
7,2012-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False,True
8,2012-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga,False,True
9,2012-06-25,Holiday,Local,Machala,Fundacion de Machala,False,True


In [33]:
holidays_events['holiday'] = True

In [35]:
national_holidays = holidays_events.loc[(holidays_events['locale'] == 'National') & (holidays_events['transferred'] == False)]

regional_holidays = holidays_events.loc[(holidays_events['locale'] == 'Regional') & (holidays_events['transferred'] == False)]

local_holidays = holidays_events.loc[(holidays_events['locale'] == 'Local') & (holidays_events['transferred'] == False)]

In [37]:
local_holidays = local_holidays.rename(index=str, columns={"locale_name": "city"})
regional_holidays = regional_holidays.rename(index=str, columns={"locale_name": "state"})

In [38]:
local_holidays.head()

,date,type,locale,city,description,transferred,holiday
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,True
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,True
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,True
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,True
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False,True


In [39]:
regional_holidays.head()

,date,type,locale,state,description,transferred,holiday
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,True
7,2012-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False,True
23,2012-11-06,Holiday,Regional,Santo Domingo de los Tsachilas,Provincializacion de Santo Domingo,False,True
24,2012-11-07,Holiday,Regional,Santa Elena,Provincializacion Santa Elena,False,True
47,2013-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,True


### Create DataFrame with all items for all stores on all dates

Lots of help from:
https://www.kaggle.com/inversion/dataframe-with-all-date-store-item-combinations/notebook

In [15]:
orig_train_index = train.index # will need later

# Convert 'date' to datetime format
# train['date'] = pd.to_datetime(train['date'])

# Bracket the dates
max_date = train['date'].max()  
min_date = train['date'].min()  
days = (max_date - min_date).days + 1

# Master list of dates
dates = [min_date + datetime.timedelta(days=x) for x in range(days)]
dates.sort()

# Master list of Items
unique_items = list(set(train['item_nbr'].unique())) # | set(test['item_nbr'].unique()))
unique_items.sort()
num_unique_items = len(unique_items)
print(num_unique_items)

# Master list of stores
unique_stores = list(set(train['store_nbr'].unique())) # | set(test['Store'].unique()))
unique_stores.sort()
num_unique_stores = len(unique_stores)
print(num_unique_stores)

# Unique Date / Store index
date_index = np.repeat(dates, num_unique_stores * num_unique_items) # num dates * num stores * num items
store_index = np.concatenate([np.repeat(unique_stores, num_unique_items)] * days)
item_index = np.concatenate([unique_items] * days * num_unique_stores)

print(len(date_index))
print(len(store_index))
print(len(item_index))

4003
54
39341484
39341484
39341484


In [16]:
start = train.index.tolist()[0]; start

106303617

In [17]:
# num of rows in date_mask
new_train_index = list(range(len(item_index)))

# Broadcast along the list
# new_train_index = np.asarray(new_train_index) + start

In [18]:
train_new = pd.DataFrame(index=new_train_index, columns=train.columns)

train_new['date'] = date_index
train_new['store_nbr'] = store_index
train_new['item_nbr'] = item_index

train_new.index.name = 'id'

In [19]:
# Set the indexes (makes it easy to insert data into new)
# This links the indexes on the queries

train_new.set_index(['date', 'store_nbr', 'item_nbr'], drop=True, inplace=True)
train.set_index(['date', 'store_nbr', 'item_nbr'], drop=True, inplace=True)

# updates df with values from other (according to index -- hence the above step)
train_new.update(train)

In [20]:
# Return the indexes back to normal
train_new.reset_index(inplace=True)
train.reset_index(inplace=True)

train.set_index(orig_train_index, inplace=True)

# Fill the created unit sales with zero
train_new['unit_sales'].fillna(0, inplace=True)
train_new['onpromotion'].fillna(False, inplace=True)

In [21]:
train_new.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2017-02-15,1,96995,0.0,False
1,2017-02-15,1,99197,0.0,False
2,2017-02-15,1,103501,0.0,False
3,2017-02-15,1,103520,0.0,False
4,2017-02-15,1,103665,2.0,False
5,2017-02-15,1,105574,7.0,False
6,2017-02-15,1,105575,12.0,False
7,2017-02-15,1,105576,0.0,False
8,2017-02-15,1,105577,0.0,False
9,2017-02-15,1,105693,0.0,False


In [22]:
# save train_new
train_new.to_feather(f'{PATH}train_six_months_full_data')

### Join Tables

In [9]:
train_new = pd.read_feather(f'{PATH}train_six_months_full_data')

In [23]:
# Need to convert the 'date' columns in transactions, oil, holidays_events, and test to match

transactions['date'] = pd.to_datetime(transactions['date'])
oil['date'] = pd.to_datetime(oil['date'])
holidays_events['date'] = pd.to_datetime(holidays_events['date'])
test['date'] = pd.to_datetime(test['date'])

In [24]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

In [25]:
joined = join_df(train_new, items, "item_nbr")
joined_test = join_df(test, items, "item_nbr")
len(joined[joined.item_nbr.isnull()]),len(joined_test[joined_test.item_nbr.isnull()])

(0, 0)

In [26]:
joined = join_df(joined, stores, "store_nbr")
joined_test = join_df(joined_test, stores, "store_nbr")
len(joined[joined.cluster.isnull()]),len(joined_test[joined_test.cluster.isnull()])

(0, 0)

In [40]:
joined.to_feather(f'{PATH}joined')
joined_test.to_feather(f'{PATH}joined_test')

In [67]:
joined.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type,cluster,transactions,dcoilwtico
0,2017-02-15,1,96995,0.0,False,GROCERY I,1093,0,Quito,Pichincha,D,13,1780.0,53.11
1,2017-02-15,1,99197,0.0,False,GROCERY I,1067,0,Quito,Pichincha,D,13,1780.0,53.11
2,2017-02-15,1,103501,0.0,False,CLEANING,3008,0,Quito,Pichincha,D,13,1780.0,53.11
3,2017-02-15,1,103520,0.0,False,GROCERY I,1028,0,Quito,Pichincha,D,13,1780.0,53.11
4,2017-02-15,1,103665,2.0,False,BREAD/BAKERY,2712,1,Quito,Pichincha,D,13,1780.0,53.11


In [69]:
joined_test.head()

,id,date,store_nbr,item_nbr,onpromotion,family,class,perishable,city,state,type,cluster,dcoilwtico
0,125497040,2017-08-16,1,96995,False,GROCERY I,1093,0,Quito,Pichincha,D,13,46.8
1,125497041,2017-08-16,1,99197,False,GROCERY I,1067,0,Quito,Pichincha,D,13,46.8
2,125497042,2017-08-16,1,103501,False,CLEANING,3008,0,Quito,Pichincha,D,13,46.8
3,125497043,2017-08-16,1,103520,False,GROCERY I,1028,0,Quito,Pichincha,D,13,46.8
4,125497044,2017-08-16,1,103665,False,BREAD/BAKERY,2712,1,Quito,Pichincha,D,13,46.8


In [30]:
joined = join_df(joined, transactions, ['date', 'store_nbr'])
# no transaction data for test dataset...
# joined_test = join_df(joined_test, transactions, ['date', 'store_nbr'])
len(joined[joined.transactions.isnull()]) #,len(joined_test[joined_test.transactions.is_null()])

256192

In [31]:
joined = join_df(joined, oil, 'date')
joined_test = join_df(joined_test, oil, 'date')
len(joined[joined.dcoilwtico.isnull()]),len(joined_test[joined_test.dcoilwtico.isnull()])

(12321234, 842616)

In [ ]:
joined = join_df(joined, national_holidays[['date', 'holiday']], 'date')
joined_test = join_df(joined_test, national_holidays[['date', 'holiday']], 'date')

joined = join_df(joined, regional_holidays[['date', 'state', 'holiday']], ['date', 'state'])
joined_test = join_df(joined_test, regional_holidays[['date', 'state', 'holiday']], ['date', 'state'])

joined = join_df(joined, local_holidays[['date', 'city', 'holiday']], ['date', 'city'])
joined_test = join_df(joined_test, local_holidays[['date', 'city', 'holiday']], ['date', 'city'])

len(joined[joined.holiday.isnull()]),len(joined_test[joined_test.holiday.isnull()])

In [68]:
joined_test.drop(['holiday', 'holiday_y'], axis=1, inplace=True)

In [4]:
joined = pd.read_feather(f'{PATH}joined')
joined_test = pd.read_feather(f'{PATH}joined_test')

In [9]:
sample = joined.sample(1000000)

In [14]:
joined.reset_index(inplace=True)

In [ ]:
joined.drop('index', inplace=True)

In [11]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 22871099 to 41202929
Data columns (total 13 columns):
date           1000000 non-null datetime64[ns]
store_nbr      1000000 non-null int64
item_nbr       1000000 non-null int64
unit_sales     1000000 non-null float64
onpromotion    1000000 non-null bool
family         1000000 non-null object
class          1000000 non-null int64
perishable     1000000 non-null int64
city           1000000 non-null object
state          1000000 non-null object
type           1000000 non-null object
cluster        1000000 non-null int64
dcoilwtico     688875 non-null float64
dtypes: bool(1), datetime64[ns](1), float64(2), int64(5), object(4)
memory usage: 100.1+ MB


In [12]:
sample.to_feather(f'{PATH}sample_million')

ValueError: feather does not support serializing a non-default index for the index; you can .reset_index() to make the index into column(s)

In [6]:
add_datepart(joined, "date", drop=False)
# add_datepart(joined_test, "date", drop=False)

In [12]:
joined.head()

,index,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,0,2016-08-15,1,96995,0.0,False,GROCERY I,1093,0,Quito,...,15,0,228,False,False,False,False,False,False,1471219200
1,1,2016-08-15,1,99197,0.0,False,GROCERY I,1067,0,Quito,...,15,0,228,False,False,False,False,False,False,1471219200
2,2,2016-08-15,1,103501,0.0,False,CLEANING,3008,0,Quito,...,15,0,228,False,False,False,False,False,False,1471219200
3,3,2016-08-15,1,103520,0.0,False,GROCERY I,1028,0,Quito,...,15,0,228,False,False,False,False,False,False,1471219200
4,4,2016-08-15,1,103665,1.0,False,BREAD/BAKERY,2712,1,Quito,...,15,0,228,False,False,False,False,False,False,1471219200


In [9]:
joined.columns

Index(['index', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion',
       'family', 'class', 'perishable', 'city', 'state', 'type', 'cluster',
       'dcoilwtico', 'Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
       'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start',
       'Is_year_end', 'Is_year_start', 'Elapsed'],
      dtype='object')

### Features

In [13]:
cat_vars = ['store_nbr', 'item_nbr', 'onpromotion',
       'family', 'class', 'perishable', 'city', 'state',
       'type', 'cluster', 'Month', 'Week', 'Day', 'Dayofweek']

contin_vars = ['unit_sales', 'dcoilwtico']

n = len(joined); n

79767504

In [17]:
joined = joined[cat_vars + contin_vars + ['date']].copy()

MemoryError: 